In [1]:
!pip install python-binance

In [2]:
from binance.client import Client
from binance import BinanceSocketManager
import pandas as pd
import datetime as dt

In [3]:
api_key = "imuSkmeAI4PpxNERZRcnzx6nEkReEhuLedZphxZNphhnUPLR4QH61aXfjnBqUtfH"
api_secret = "56AvdFdOWWoRF20rh6Knq8awDYM145luMDH0XvDMkSD6nffbRwYTCrsxwJmyVLK1"

In [4]:
client = Client(api_key, api_secret)

In [5]:
#Get the exchange info, which contains information about all trading pairs
#exchange_info = client.get_exchange_info()

#Extract the symbols of all trading pairs
#symbols = [symbol['symbol'] for symbol in exchange_info['symbols'] if "BRL" in symbol['symbol']]

#Filter out only the base cryptocurrencies (e.g., BTC, ETH)
#symbols = list(symbol.split(' ')[0] for symbol in symbols)
symbols = ["BTCBRL","ETHBRL","SOLBRL","AXSBRL","LUNABRL"]
print(f"{len(symbols)}\n")
print(symbols)

5

['BTCBRL', 'ETHBRL', 'SOLBRL', 'AXSBRL', 'LUNABRL']


In [6]:
# Function to fetch historical cryptocurrency data
def fetch_binance_cryptocurrency_data(symbol, start_date, end_date, interval='1d'):
    klines = client.get_historical_klines(symbol, interval, start_date, end_date)

    # Convert data to a Pandas DataFrame
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  # Convert timestamp to datetime

    return df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

database = {}
present_day = dt.date.today()
start_date = '1900-01-01'
end_date = f"{present_day}"
for ticker in symbols:
    data = fetch_binance_cryptocurrency_data(ticker, start_date, end_date)
    database[ticker] = data.close.values.astype(float)
    
database

{'BTCBRL': array([ 63921.,  64068.,  65314., ..., 329697., 333253., 335356.]),
 'ETHBRL': array([ 2227.54,  2246.94,  2298.3 , ..., 19050.4 , 19296.22, 19659.28]),
 'SOLBRL': array([ 189. ,  181.3,  182.1,  135.7,  135.3,  153.7,  154.9,  143.9,
         147.6,  157.5,  163.6,  168.8,  178.1,  169.6,  172.7,  174.9,
         173.8,  169.9,  178.6,  193.6,  176.8,  176.2,  168.8,  170.1,
         161.1,  152. ,  160.6,  147.2,  136.6,  138.4,  137.9,  129.4,
         123.4,  139.8,  145.9,  149.1,  150. ,  147. ,  147.9,  146.3,
         143.6,  159.6,  168.9,  191. ,  178.5,  172.8,  178.2,  186.8,
         195.6,  207.2,  206.8,  198.5,  204. ,  212.9,  220. ,  217.8,
         237. ,  233.2,  282.2,  331.1,  343.4,  394. ,  393. ,  424.5,
         400.6,  393.9,  410.8,  375.2,  378.9,  397.7,  459.9,  504.2,
         492.3,  578.2,  565.6,  575. ,  668.1,  763.5,  726.7,  738.5,
         857. ,  919.9, 1016.9,  990. ,  955.9,  951. ,  924.3,  889. ,
         834.1,  832. ,  805.4,  7

In [7]:
!pip install --upgrade h5py

In [8]:
import numpy as np
!pip install tensorflow

In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN #, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import mean_squared_error

In [39]:
def predict(price: list):
    # Prepare data
    lookback_window = 11  # Adjust this window size as needed
    X = []
    y = []
    for i in range(len(price) - lookback_window):
        X.append(price[i:i+lookback_window])
        y.append(price[i+lookback_window])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], 1)

        
    # Split data
    #split = int(0.99 * len(X))
    #size = len(X)-2
    #X_train, X_test = X[:size], X[size:]
    #y_train, y_test = y[:size], y[size:]
    
    # Create & train model
    model = Sequential([
        SimpleRNN(64, activation='relu', input_shape=(lookback_window, 1), return_sequences=True),
        SimpleRNN(64, activation='relu', return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=100, verbose=0)

    # Predict & evaluate
    X_test = y[-11:]
    pred1 = model.predict(X_test.reshape(1,11,1))
    new_x_test = pred1[0][0]
    X_test = np.append(X_test,new_x_test)
    pred2 = model.predict(X_test[-11:].reshape(1,11,1))
    
    #loss = model.evaluate(X_test, y_test, verbose=0)
    return y[-1] < pred1[0][0] < pred2[0][0], y[-1] ,pred1[0][0],pred2[0][0]

In [40]:
status_list = []
last_price_list,prediction_1_list,prediction_2_list = [],[],[]
#final_proft = 0
#true_proft = 0
#true_status = []
#total_stock_price = 0
for ticker in symbols:
    price = database[ticker]
    status,last_price,prediction_1,prediction_2 = predict(price)
    status_list.append(status)
    last_price_list.append(last_price)
    prediction_1_list.append(prediction_1)
    prediction_2_list.append(prediction_2)
    #if status:
        #final_proft += proft
        #total_stock_price += current_price
    #if proft > 0:
        #true_status.append(True)
        #true_proft += proft
    #if proft <= 0:
        #true_status.append(False)

print('\nmodel status:', status_list)#, '\ntrue status:', true_status)
#print('\nfinal proft:', final_proft, '\ntrue proft:', true_proft)
#print('\n% increased:', (total_stock_price + final_proft) / total_stock_price)

1/1 [==============================] - 0s 16ms/step

model status: [False, True, False, True, False]


In [41]:
dic = {"Ticker":[],"Act":[],"LastPrice":[],"Pred1":[],"Pred2":[]}
for ticker,act_status,lst_price,pred_1,pred_2 in zip(symbols,status_list,last_price_list,prediction_1_list,prediction_2_list):
    dic["Ticker"].append(ticker)
    dic["Act"].append(act_status)
    dic["LastPrice"].append(lst_price)
    dic["Pred1"].append(pred_1)
    dic["Pred2"].append(pred_2)
df = pd.DataFrame(dic)
display(df)

,Ticker,Act,LastPrice,Pred1,Pred2
0,BTCBRL,False,335356.0000,334449.000000,332891.125000
1,ETHBRL,True,19659.2800,20156.564453,20620.845703
2,SOLBRL,False,727.3000,698.023254,674.421936
3,AXSBRL,True,38.3600,39.590557,40.963352
4,LUNABRL,False,0.0003,-2.044836,12.862169


In [13]:
#csv_file = df.to_csv(f"Prediction_{present_day}.csv",index=False)

# Test

In [14]:
"""
def predict(price: list):
    # Prepare data
    lookback_window = 11  # Adjust this window size as needed
    X = []
    y = []
    for i in range(len(price) - lookback_window):
        X.append(price[i:i+lookback_window])
        y.append(price[i+lookback_window])
    X = np.array(X)
    y = np.array(y)
    X = X.reshape(X.shape[0], X.shape[1], 1)

        
    # Split data
    split = int(0.99 * len(X))
    size = len(X)-2
    X_train, X_test = X[:size], X[size:]
    y_train, y_test = y[:size], y[size:]
    
    # Create & train model
    model = Sequential([
        SimpleRNN(64, activation='relu', input_shape=(lookback_window, 1), return_sequences=True),
        SimpleRNN(64, activation='relu', return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=100, verbose=0)

    # Predict & evaluate
    predictions = model.predict(X_test)
    loss = model.evaluate(X_test, y_test, verbose=0)
    return y_train[-1] < predictions[0][0] < predictions[1][0] , y_test[0]-y_train[-1] , y_train[-1]
"""

"\ndef predict(price: list):\n    # Prepare data\n    lookback_window = 11  # Adjust this window size as needed\n    X = []\n    y = []\n    for i in range(len(price) - lookback_window):\n        X.append(price[i:i+lookback_window])\n        y.append(price[i+lookback_window])\n    X = np.array(X)\n    y = np.array(y)\n    X = X.reshape(X.shape[0], X.shape[1], 1)\n\n        \n    # Split data\n    split = int(0.99 * len(X))\n    size = len(X)-2\n    X_train, X_test = X[:size], X[size:]\n    y_train, y_test = y[:size], y[size:]\n    \n    # Create & train model\n    model = Sequential([\n        SimpleRNN(64, activation='relu', input_shape=(lookback_window, 1), return_sequences=True),\n        SimpleRNN(64, activation='relu', return_sequences=False),\n        Dense(1)\n    ])\n    model.compile(optimizer='adam', loss='mse')\n    model.fit(X_train, y_train, epochs=100, verbose=0)\n\n    # Predict & evaluate\n    predictions = model.predict(X_test)\n    loss = model.evaluate(X_test, y_test

In [15]:
"""
status_list = []
final_proft = 0
true_proft = 0
true_status = []
total_stock_price = 0
for ticker in symbols:
    price = database[ticker]
    status, proft,current_price = predict(price)
    status_list.append(status)
    if status:
        final_proft += proft
        total_stock_price += current_price
    if proft > 0:
        true_status.append(True)
        true_proft += proft
    if proft <= 0:
        true_status.append(False)

print('\nmodel status:', status_list, '\ntrue status:', true_status)
print('\nfinal proft:', final_proft, '\ntrue proft:', true_proft)
print('\n% increased:', (total_stock_price + final_proft) / total_stock_price)
"""

"\nstatus_list = []\nfinal_proft = 0\ntrue_proft = 0\ntrue_status = []\ntotal_stock_price = 0\nfor ticker in symbols:\n    price = database[ticker]\n    status, proft,current_price = predict(price)\n    status_list.append(status)\n    if status:\n        final_proft += proft\n        total_stock_price += current_price\n    if proft > 0:\n        true_status.append(True)\n        true_proft += proft\n    if proft <= 0:\n        true_status.append(False)\n\nprint('\nmodel status:', status_list, '\ntrue status:', true_status)\nprint('\nfinal proft:', final_proft, '\ntrue proft:', true_proft)\nprint('\n% increased:', (total_stock_price + final_proft) / total_stock_price)\n"

In [16]:
"""
dic = {"Ticker":[],"Act":[],"True_Act":[]}
for ticker,act_status,true_act in zip(symbols,status_list,true_status):   
    dic["Ticker"].append(ticker)
    dic["Act"].append(act_status)
    dic["True_Act"].append(true_act)
df = pd.DataFrame(dic)
display(df)
"""

'\ndic = {"Ticker":[],"Act":[],"True_Act":[]}\nfor ticker,act_status,true_act in zip(symbols,status_list,true_status):   \n    dic["Ticker"].append(ticker)\n    dic["Act"].append(act_status)\n    dic["True_Act"].append(true_act)\ndf = pd.DataFrame(dic)\ndisplay(df)\n'